In [1]:
import cv2
import numpy as np

In [2]:
SHOW_TIMER = 1000

In [3]:
frame = cv2.imread("../frames/masked_frame.png")
frame = cv2.imread("../frames/frame_208900_6963300.0.jpg")

hsv_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)

In [4]:
cv2.imshow("frame", frame)
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows()

In [5]:
cv2.imshow("Vue", hsv_frame[:,:,2])
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [6]:
cv2.imshow("Saturation", hsv_frame[:,:,1])
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [7]:
cv2.imshow("Hue", hsv_frame[:,:,0])
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [8]:
COLORS = ["blue", "red", "pink", "brown", "yellow", "green", "white", "black"]

In [9]:
MAX_BLUE = (25, 255, 255)
MIN_BLUE = (10, 0, 30)
mask_blue = cv2.inRange(hsv_frame, MIN_BLUE, MAX_BLUE) # 

In [10]:
cv2.imshow("mask blue", mask_blue) 
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows()

In [11]:
MAX_RED = (130, 255, 200)
MIN_RED = (110, 0, 0)
mask_red = cv2.inRange(hsv_frame, MIN_RED, MAX_RED) #

In [12]:
cv2.imshow("mask red", mask_red) 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
MAX_PINK = (130, 255, 255)
MIN_PINK = (110, 0, 200)
mask_pink = cv2.inRange(hsv_frame, MIN_PINK, MAX_PINK)

In [14]:
cv2.imshow("mask pink", mask_pink) 
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows()

In [15]:
MAX_BROWN = (110, 255, 150)
MIN_BROWN = (90, 0, 0)
mask_brown = cv2.inRange(hsv_frame, MIN_BROWN, MAX_BROWN)

In [16]:
cv2.imshow("mask brown", mask_brown) 
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows()

In [17]:
MAX_YELLOW = (100, 255, 255)
MIN_YELLOW = (90, 0, 150)
mask_yellow = cv2.inRange(hsv_frame, MIN_YELLOW, MAX_YELLOW)

In [18]:
cv2.imshow("mask yellow", mask_yellow) 
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows()

In [19]:
MAX_GREEN = (50, 255, 255)
MIN_GREEN = (30, 0, 0)
mask_green = cv2.inRange(hsv_frame, MIN_GREEN, MAX_GREEN) 

In [20]:
cv2.imshow("mask green", mask_green) 
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows()

In [21]:
MAX_WHITE = (90, 255, 255)
MIN_WHITE = (70, 0, 150)
mask_white = cv2.inRange(hsv_frame, MIN_WHITE, MAX_WHITE) 

In [22]:
cv2.imshow("mask white", mask_white) 
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows()

In [23]:
MAX_BLACK = (255, 255, 50)
MIN_BLACK = (0, 0, 0)
mask_black = cv2.inRange(hsv_frame, MIN_BLACK, MAX_BLACK) 


In [24]:
cv2.imshow("mask black", mask_black) 
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows()

In [25]:
def get_neighbours(point):
    x, y = point
    return [[x,y], [x+1,y], [x-1,y], [x,y+1], [x,y-1]]

def merge_groups(groups, ignore_groups):
    ignore_groups = []
    for current_index, current_group in enumerate(groups):
        if current_index in ignore_groups: continue
        for point in current_group:
            neighbours = get_neighbours(point)
            for index, group in enumerate(groups):
                if index == current_index: continue
                #print(group)
                exists = any(neighbour in group for neighbour in neighbours)
                if exists:
                    groups[index] = groups[index]+groups[current_index]
                    groups.pop(current_index)
                    print("merged", index, current_index, "=", len(groups))
                    print(groups)
                    return groups, True, ignore_groups
        ignore_groups += [current_index]
    print(groups)            
    return groups, False, ignore_groups
    
def get_largest_group(mask):
    candidates = np.where(mask == 255)
    
    groups = []
    for px, py in zip(candidates[0], candidates[1]):
        added = False
        neighbours = get_neighbours((px, py))
        # check if belongs to an already created group
        for index, group in enumerate(groups):
            exists = any(elem in group for elem in neighbours)
            if exists:
                groups[index] = groups[index]+[[px, py]]
                added = True
        if not added:
            print("created new")
            groups += [[[px, py]]]

    #merge groups
    ignore_groups = []
    merged=True
    while merged:
        groups, merged, ignore_groups = merge_groups(groups, ignore_groups)
    print("main", groups)
    groups = sorted(groups, key=len, reverse=True)
    if len(groups)==0: 
        print("No ball found")
        return None
    return groups[0]
    

def get_n_largest_groups(mask, n=1): # this is an inplace replacement for the func above, but the return type is list!
    candidates = np.where(mask == 255)
    
    groups = []
    for px, py in zip(candidates[0], candidates[1]):
        added = False
        neighbours = get_neighbours((px, py))
        # check if belongs to an already created group
        for index, group in enumerate(groups):
            exists = any(elem in group for elem in neighbours)
            if exists:
                groups[index] = groups[index]+[[px, py]]
                added = True
        if not added:
            print("created new")
            groups += [[[px, py]]]

    #merge groups
    ignore_groups = []
    merged=True
    while merged and len(groups)>n:
        groups, merged, ignore_groups = merge_groups(groups, ignore_groups)
    print("main", groups)
    groups = sorted(groups, key=len, reverse=True)
    if len(groups)==0: 
        print("No ball found")
        return None
    return groups[0:n]

In [26]:
COLORS

['blue', 'red', 'pink', 'brown', 'yellow', 'green', 'white', 'black']

In [27]:
raw_masks = [mask_blue, mask_red, mask_pink, mask_brown, mask_yellow, mask_green, mask_white, mask_black]
masks = []

In [28]:
"""
dilated_mask = cv2.dilate(mask_white, None, iterations=2)
largest_group = get_largest_group(dilated_mask)
new_mask = np.zeros(mask_black.shape)
for point in largest_group:
    new_mask[point[0], point[1]] = 255
cv2.imshow("test", new_mask) 
cv2.waitKey(0)
cv2.destroyAllWindows()
"""

'\ndilated_mask = cv2.dilate(mask_white, None, iterations=2)\nlargest_group = get_largest_group(dilated_mask)\nnew_mask = np.zeros(mask_black.shape)\nfor point in largest_group:\n    new_mask[point[0], point[1]] = 255\ncv2.imshow("test", new_mask) \ncv2.waitKey(0)\ncv2.destroyAllWindows()\n'

In [29]:
# Dilate all masks and get largest patch:
for index, mask in enumerate(raw_masks):
    if index==1: 
        masks.append(mask)
        continue # Not parsing red as there are multiple red balls
    eroded_mask = cv2.erode(mask, None, iterations=2)
    dilated_mask = cv2.dilate(eroded_mask, None, iterations=2+2)
    cv2.imshow(COLORS[index], dilated_mask) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    largest_group = get_largest_group(dilated_mask)
    if largest_group == None: continue
    new_mask = np.zeros(mask.shape)
    for point in largest_group:
        new_mask[point[0], point[1]] = 255
    masks.append(new_mask)
    cv2.imshow(COLORS[index], new_mask) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

created new
created new
created new
merged 1 0 = 2
[[[271, 633], [271, 634], [271, 635], [271, 636], [271, 637], [271, 638], [271, 639], [271, 640], [271, 641], [271, 642], [271, 643], [271, 644], [271, 645], [271, 646], [271, 647], [271, 648], [271, 649], [272, 633], [272, 634], [272, 635], [272, 636], [272, 637], [272, 638], [272, 639], [272, 640], [272, 641], [272, 642], [272, 643], [272, 644], [272, 645], [272, 646], [272, 647], [272, 648], [272, 649], [273, 633], [273, 634], [273, 635], [273, 636], [273, 637], [273, 638], [273, 639], [273, 640], [273, 641], [273, 642], [273, 643], [273, 644], [273, 645], [273, 646], [273, 647], [273, 648], [273, 649], [274, 633], [274, 634], [274, 635], [274, 636], [274, 637], [274, 638], [274, 639], [274, 640], [274, 641], [274, 642], [274, 643], [274, 644], [274, 645], [274, 646], [274, 647], [274, 648], [274, 649], [275, 633], [275, 634], [275, 635], [275, 636], [275, 637], [275, 638], [275, 639], [275, 640], [275, 641], [275, 642], [275, 643],

created new
created new
merged 1 0 = 1
[[[131, 632], [131, 633], [131, 634], [131, 635], [131, 636], [131, 637], [131, 638], [131, 639], [131, 640], [131, 641], [131, 642], [131, 643], [131, 644], [131, 645], [131, 646], [132, 632], [132, 633], [132, 634], [132, 635], [132, 636], [132, 637], [132, 638], [132, 639], [132, 640], [132, 641], [132, 642], [132, 643], [132, 644], [132, 645], [132, 646], [133, 632], [133, 633], [133, 634], [133, 635], [133, 636], [133, 637], [133, 638], [133, 639], [133, 640], [133, 641], [133, 642], [133, 643], [133, 644], [133, 645], [133, 646], [134, 632], [134, 633], [134, 634], [134, 635], [134, 636], [134, 637], [134, 638], [134, 639], [134, 640], [134, 641], [134, 642], [134, 643], [134, 644], [134, 645], [134, 646], [135, 632], [135, 633], [135, 634], [135, 635], [135, 636], [135, 637], [135, 638], [135, 639], [135, 640], [135, 641], [135, 642], [135, 643], [135, 644], [135, 645], [135, 646], [136, 632], [136, 633], [136, 634], [136, 635], [136, 636],

created new
[[[54, 390], [54, 391], [54, 392], [54, 393], [54, 394], [54, 395], [54, 396], [54, 397], [54, 398], [55, 390], [55, 391], [55, 392], [55, 393], [55, 394], [55, 395], [55, 396], [55, 397], [55, 398], [56, 390], [56, 391], [56, 392], [56, 393], [56, 394], [56, 395], [56, 396], [56, 397], [56, 398], [57, 390], [57, 391], [57, 392], [57, 393], [57, 394], [57, 395], [57, 396], [57, 397], [57, 398], [58, 390], [58, 391], [58, 392], [58, 393], [58, 394], [58, 395], [58, 396], [58, 397], [58, 398], [58, 399], [58, 400], [58, 401], [58, 402], [58, 403], [59, 390], [59, 391], [59, 392], [59, 393], [59, 394], [59, 395], [59, 396], [59, 397], [59, 398], [59, 399], [59, 400], [59, 401], [59, 402], [59, 403], [60, 390], [60, 391], [60, 392], [60, 393], [60, 394], [60, 395], [60, 396], [60, 397], [60, 398], [60, 399], [60, 400], [60, 401], [60, 402], [60, 403], [61, 390], [61, 391], [61, 392], [61, 393], [61, 394], [61, 395], [61, 396], [61, 397], [61, 398], [61, 399], [61, 400], [61, 40

created new
created new
created new
created new
merged 1 0 = 3
[[[162, 764], [162, 765], [162, 766], [162, 767], [162, 768], [162, 769], [162, 770], [162, 771], [162, 772], [162, 773], [162, 774], [162, 775], [162, 776], [162, 777], [162, 778], [163, 764], [163, 765], [163, 766], [163, 767], [163, 768], [163, 769], [163, 770], [163, 771], [163, 772], [163, 773], [163, 774], [163, 775], [163, 776], [163, 777], [163, 778], [163, 779], [164, 764], [164, 765], [164, 766], [164, 767], [164, 768], [164, 769], [164, 770], [164, 771], [164, 772], [164, 773], [164, 774], [164, 775], [164, 776], [164, 777], [164, 778], [164, 779], [165, 764], [165, 765], [165, 766], [165, 767], [165, 768], [165, 769], [165, 770], [165, 771], [165, 772], [165, 773], [165, 774], [165, 775], [165, 776], [165, 777], [165, 778], [165, 779], [166, 764], [166, 765], [166, 766], [166, 767], [166, 768], [166, 769], [166, 770], [166, 771], [166, 772], [166, 773], [166, 774], [166, 775], [166, 776], [166, 777], [166, 778],

created new
created new
created new
merged 2 0 = 2
[[[516, 499], [516, 500], [516, 501], [516, 502], [516, 503], [516, 504], [516, 505], [516, 506], [516, 507], [517, 499], [517, 500], [517, 501], [517, 502], [517, 503], [517, 504], [517, 505], [517, 506], [517, 507], [518, 499], [518, 500], [518, 501], [518, 502], [518, 503], [518, 504], [518, 505], [518, 506], [518, 507], [519, 499], [519, 500], [519, 501], [519, 502], [519, 503], [519, 504], [519, 505], [519, 506], [519, 507], [520, 499], [520, 500], [520, 501], [520, 502], [520, 503], [520, 504], [520, 505], [520, 506], [520, 507], [521, 499], [521, 500], [521, 501], [521, 502], [521, 503], [521, 504], [521, 505], [521, 506], [521, 507], [522, 499], [522, 500], [522, 501], [522, 502], [522, 503], [522, 504], [522, 505], [522, 506], [522, 507], [523, 499], [523, 500], [523, 501], [523, 502], [523, 503], [523, 504], [523, 505], [523, 506], [523, 507], [524, 499], [524, 500], [524, 501], [524, 502], [524, 503], [524, 504], [524, 505],

created new
created new
created new


KeyboardInterrupt: 

In [ ]:
for index, mask in enumerate(masks):
    cv2.imshow(COLORS[index], mask) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Now, in `masks` there are the masks for all colored balls 

In [ ]:
def matches_window(frame, lower_mask, color_min_upper, color_max_upper):
    lower = lower_mask#cv2.inRange(frame, color_min_lower, color_max_lower)
    upper = cv2.inRange(frame, color_min_upper, color_max_upper)

    cv2.imshow("lower", lower) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    cv2.imshow("upper", upper) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    lower_matched_x, lower_matched_y = np.where(lower==255)

    matchings = []
    for lower_candidate_x, lower_candidate_y in zip(lower_matched_x, lower_matched_y):
        if lower[lower_candidate_x, lower_candidate_y] > 0:
            if upper[lower_candidate_x-1, lower_candidate_y] == 255:
                matchings.append((lower_candidate_x-1, lower_candidate_y))
        else: print("image window out of bounds")
    return matchings
            

In [ ]:
def visualize_points(image, points, color):
    # Takes a greyscale image and paints some points in color on top
    image = image.copy()
    image=np.zeros((image.shape[0], image.shape[1], 3))#.astype(int)
    #image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    for x,y in points:
        image[x,y] = color
    return image

In [ ]:
highlight_masks = []

In [ ]:
MIN_HIGHLIGHT = (0, 0, 245) # for the black highlight, this should be 145
MIN_HIGHLIGHT = (0, 0, 145)
MAX_HIGHLIGHT = (255, 255, 255) 
for mask in masks:
    matchings = matches_window(hsv_frame, mask, MIN_HIGHLIGHT, MAX_HIGHLIGHT)
    matchings_visualized = visualize_points(mask, matchings, (0,0,255))
    highlight_masks.append(matchings_visualized)
    cv2.imshow("matchings", matchings_visualized) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [ ]:
for index, highlight in enumerate(highlight_masks):
    cv2.imshow(COLORS[index], highlight) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
cv2.imshow("matchings", highlight_masks[-1]) 
cv2.waitKey(0)
cv2.destroyAllWindows()

Now, detect the actual highlight and get their center:

Idea: Dilute these findings and get the brightest pixels in them, for red, then group these brightest spots into 15 groups.

In [ ]:
highlight_masks_diluted = []
for index, highlight in enumerate(highlight_masks):
    eroded_mask = cv2.erode(highlight, None, iterations=0)
    dilated_mask = cv2.dilate(eroded_mask, None, iterations=4)
    highlight_masks_diluted.append(dilated_mask)
    opacity = 0.2
    cv2.imshow(COLORS[index], cv2.addWeighted(frame.astype(np.uint8), opacity, dilated_mask.astype(np.uint8), 1, 0)) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
results = []
for index, highlight in enumerate(highlight_masks_diluted):
    print(COLORS[index])
    search_mask = np.where(highlight[:,:,2] != 255)
    # at these locations, check the hsv frame for bright pixels

    black_pixels = np.where(highlight[:,:,2] != 255)
    cropped_frame = hsv_frame.copy()
    cropped_frame[black_pixels] = [0, 0, 0]

    cv2.imshow(f"Search Area {COLORS[index]}", cropped_frame) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    MIN_HIGHLIGHT = (0, 0, 245)
    MAX_HIGHLIGHT = (255, 120, 255)
    if COLORS[index] == "black":
        MIN_HIGHLIGHT = (0, 0, 145)
    if COLORS[index] == "yellow":
        MAX_HIGHLIGHT = (255, 200, 255)
    if COLORS[index] == "pink":
        MAX_HIGHLIGHT = (255, 70, 255)

    highlight_pixels = cv2.inRange(cropped_frame, MIN_HIGHLIGHT, MAX_HIGHLIGHT)
    
    highlight_coordinates = np.where(highlight_pixels > 0)
    print(len(highlight_coordinates[0]))

    cv2.imshow(f"matchings {COLORS[index]}", highlight_pixels) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    if COLORS[index] == "red":
        largest_groups = get_n_largest_groups(highlight_pixels, n=15)
        # get mean per group
        print("\n\n\nred")
        highlight_mean = []
        new_mask = np.zeros(highlight_pixels.shape)
        for group in largest_groups:
            group_mean = np.mean(group, axis=0)
            highlight_mean.append(group_mean)
            print(group_mean)
            for point in group:
                new_mask[point[0], point[1]] = 255
                highlight_pixels = new_mask
        
        
        cv2.imshow(f"matchings {COLORS[index]}", new_mask) 
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    else:
        highlight_mean = np.mean(highlight_coordinates, axis=1)
        print(highlight_mean)
    
    results.append((COLORS[index], highlight_mean, highlight_pixels))
    


In [ ]:
cv2.imshow("result red", results[1][2])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# for all ball colors (if red x 15) do
# move the window over candiate pixels or whole frame
# get hte mean of the highlight pixels
# return [(color, highlight_coord, image_mask), ()]

Implemented as a function:

In [57]:
import numpy as np
import cv2
frame = cv2.imread("../frames/masked_frame.png")

# NOTE: Hard-coded vaues:
# Colors of the balls
COLORS = ["blue", "red", "pink", "brown", "yellow", "green", "white", "black"]
# Color of the blue ball
MAX_BLUE = (25, 255, 255)
MIN_BLUE = (10, 0, 30)
# Color of the red balls
MAX_RED = (130, 255, 190)
MIN_RED = (110, 170, 0)
# Color of the pink ball
MAX_PINK = (130, 255, 255)
MIN_PINK = (110, 0, 200)
# Color of the brown ball
MAX_BROWN = (110, 255, 150)
MIN_BROWN = (90, 0, 0)
# Color of the yellow ball
MAX_YELLOW = (100, 255, 255)
MIN_YELLOW = (90, 0, 150)
# Color of the green ball
MAX_GREEN = (50, 255, 255)
MIN_GREEN = (30, 0, 0)
# Color of the white ball
MAX_WHITE = (90, 255, 255)
MIN_WHITE = (70, 0, 150)
# Color of the black ball
MAX_BLACK = (50, 255, 50) # TODO REMOVE THIS
MIN_BLACK = (1, 0, 0)
MAX_BLACK = (255, 255, 50)
MIN_BLACK = (0, 1, 0)
# Color of the highlight/specular
MIN_HIGHLIGHT = (0, 0, 245)
MAX_HIGHLIGHT_REGION = (255, 255, 255) # This value is used to detect the larger specular area so is more tolerant
MAX_HIGHLIGHT = (255, 120, 255) 
# Color of the highlight/specular on different balls:
MIN_HIGHLIGHT_BLACK = (0, 0, 145)
MAX_HIGHLIGHT_YELLOW = (255, 200, 255)
MAX_HIGHLIGHT_PINK = (255, 70, 255)
# Number of Iterations to erode the initial ball masks
ERODE_ITERATIONS = 2
ERODE_ITERATIONS_BLACK = 4
# Number of Iterations to dilate the initial ball masks
DILATE_ITERATIONS = 2

In [58]:
def get_neighbours(point):
    """
    Returns the neighbour coordinates of a pixel ignoring diagonals
    """
    x, y = point
    return [[x,y], [x+1,y], [x-1,y], [x,y+1], [x,y-1]]

def merge_groups(groups, ignore_groups):
    """
    Takes a list of lists containing coordinates of pixels.
    These lists are called groups.
    Merges 2 groups if they contain neighbouring pixels.
    """
    ignore_groups = [] # ignore already processed groups to improve performance
    for current_index, current_group in enumerate(groups): 
        if current_index in ignore_groups: continue
        for point in current_group:
            neighbours = get_neighbours(point)
            for index, group in enumerate(groups):
                if index == current_index: continue
                # check for every pixel if it has neighbours that belong to another group
                exists = any(neighbour in group for neighbour in neighbours)
                if exists:
                    # merge the 2 groups
                    groups[index] = groups[index]+groups[current_index]
                    groups.pop(current_index)
                    #print("merged", index, current_index, "=", len(groups))
                    return groups, True, ignore_groups
        ignore_groups += [current_index]           
    return groups, False, ignore_groups
    

def get_n_largest_groups(mask, n=1):
    """
    Given a mask, returns the n pargest patches/groups of pixels with value 255.
    """
    # NOTE: Hard-coded value of the mask's value
    candidates = np.where(mask == 255)
    
    groups = []
    for px, py in zip(candidates[0], candidates[1]):
        added = False
        neighbours = get_neighbours((px, py))
        # check if the pixel belongs to an already created group
        for index, group in enumerate(groups):
            exists = any(elem in group for elem in neighbours)
            if exists:
                groups[index] = groups[index]+[[px, py]]
                added = True
        if not added: # create new group
            groups += [[[px, py]]]

    # merge neighbouring groups, this is necessary as the previous step may have one patch into multiple groups
    ignore_groups = []
    merged=True
    while merged:# and len(groups)>n: # TODO is this robust without the n (if there are less than 15 red balls)?
        groups, merged, ignore_groups = merge_groups(groups, ignore_groups)
    groups = sorted(groups, key=len, reverse=True)
    if len(groups)==0: 
        print("No ball found")
        return None
    return groups[0:n]

def matches_window(frame, lower_mask, color_min_upper, color_max_upper):
    """
    Checks for every pixel in a given mask if the above neighbour is withing a specified range.
    Returns the coordinates of the upper pixels of such matchings.
    """
    lower = lower_mask#cv2.inRange(frame, color_min_lower, color_max_lower)
    upper = cv2.inRange(frame, color_min_upper, color_max_upper)
    
    lower_matched_x, lower_matched_y = np.where(lower==255)

    matchings = []
    for lower_candidate_x, lower_candidate_y in zip(lower_matched_x, lower_matched_y):
        if lower[lower_candidate_x, lower_candidate_y] > 0:
            if upper[lower_candidate_x-1, lower_candidate_y] == 255:
                matchings.append((lower_candidate_x-1, lower_candidate_y))
        else: print("image window out of bounds")
    return matchings
            
def visualize_points(image, points, color):
    """
    Takes an image and paints some points in specified color on top
    """
    image = image.copy()
    image=np.zeros((image.shape[0], image.shape[1], 3))#.astype(int)
    #image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    for x,y in points:
        image[x,y] = color
    return image

In [88]:
def detect_highlight(frame, version="max"):
    """
    Given a frame, returns the coordinates of the highlights/speculars on all balls.
    Version="max": returns the lower edge of the highlight
    Version="mean": returns the middle of the highlight
    Return structure:
    [(color_as_string, (x_coord, y_coord), mask_of_highlight), ...]
    for red, the second element is a list of coordinates, as there may be multiple red balls:
    [(x_coord_1, y_coord_1), (x_coord_2, y_coord_2), ...]
    """
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV) # convert frame to HSV for easier color detection
    
    mask_blue = cv2.inRange(hsv_frame, MIN_BLUE, MAX_BLUE) 
    mask_red = cv2.inRange(hsv_frame, MIN_RED, MAX_RED)
    mask_pink = cv2.inRange(hsv_frame, MIN_PINK, MAX_PINK)
    mask_brown = cv2.inRange(hsv_frame, MIN_BROWN, MAX_BROWN)
    mask_yellow = cv2.inRange(hsv_frame, MIN_YELLOW, MAX_YELLOW)
    mask_green = cv2.inRange(hsv_frame, MIN_GREEN, MAX_GREEN)
    mask_white = cv2.inRange(hsv_frame, MIN_WHITE, MAX_WHITE)
    mask_black = cv2.inRange(hsv_frame, MIN_BLACK, MAX_BLACK)

    raw_masks = [mask_blue, mask_red, mask_pink, mask_brown, mask_yellow, mask_green, mask_white, mask_black]
    masks = []
    
    # Dilate all masks and get largest patch:
    for index, mask in enumerate(raw_masks):
        erode_iterations = ERODE_ITERATIONS
        dilate_iterations = DILATE_ITERATIONS
        if index==1: # red
            masks.append(mask)
            continue # Not parsing red as there are multiple red balls
        if index==7: # black
            erode_iterations = ERODE_ITERATIONS_BLACK
        eroded_mask = cv2.erode(mask, None, iterations=erode_iterations)
        dilated_mask = cv2.dilate(eroded_mask, None, iterations=dilate_iterations+erode_iterations)

        
        largest_group = get_n_largest_groups(dilated_mask)
        if largest_group == None: continue
        largest_group=largest_group[0]
        new_mask = np.zeros(mask.shape)
        for point in largest_group:
            new_mask[point[0], point[1]] = 255
        masks.append(new_mask)

    highlight_masks = []

    for mask in masks:
        matchings = matches_window(hsv_frame, mask, MIN_HIGHLIGHT_BLACK, MAX_HIGHLIGHT_REGION)
        matchings_visualized = visualize_points(mask, matchings, (0,0,255))
        highlight_masks.append(matchings_visualized)
        #cv2.imshow("matchings", matchings_visualized) 
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()

    
    highlight_masks_diluted = []
    for index, highlight in enumerate(highlight_masks):
        eroded_mask = cv2.erode(highlight, None, iterations=0)
        dilated_mask = cv2.dilate(eroded_mask, None, iterations=4)
        highlight_masks_diluted.append(dilated_mask)
        opacity = 0.2
        #cv2.imshow(COLORS[index], cv2.addWeighted(frame.astype(np.uint8), opacity, dilated_mask.astype(np.uint8), 1, 0)) 
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
    
    
    results = []
    for index, highlight in enumerate(highlight_masks_diluted):
        #print(COLORS[index])
        search_mask = np.where(highlight[:,:,2] != 255)
        # at these locations, check the hsv frame for bright pixels
    
        black_pixels = np.where(highlight[:,:,2] != 255)
        cropped_frame = hsv_frame.copy()
        cropped_frame[black_pixels] = [0, 0, 0]

    
        
        min_highlight = MIN_HIGHLIGHT
        max_highlight = MAX_HIGHLIGHT
        if COLORS[index] == "black":
            min_highlight = MIN_HIGHLIGHT_BLACK
        if COLORS[index] == "yellow":
            max_highlight = MAX_HIGHLIGHT_YELLOW
        if COLORS[index] == "pink":
            max_highlight = MAX_HIGHLIGHT_PINK
    
        highlight_pixels = cv2.inRange(cropped_frame, min_highlight, max_highlight)
        
        highlight_coordinates = np.where(highlight_pixels > 0)
    
    
        if COLORS[index] == "red":
            largest_groups = get_n_largest_groups(highlight_pixels, n=15)
            # get mean per group
            #print("\n\n\nred")
            highlight_mean = []
            highlight_max = []
            new_mask = np.zeros(highlight_pixels.shape)
            for group in largest_groups:
                group_mean = np.mean(group, axis=0)
                highlight_mean.append(group_mean)
                highlight_max.append((np.max(np.array(group)[:, 0] ,axis=0), group_mean[1]))
                print(COLORS[index], group_mean)
                for point in group:
                    new_mask[point[0], point[1]] = 255
                    highlight_pixels = new_mask
            
        else:
            highlight_mean = np.mean(highlight_coordinates, axis=1)
            highlight_max = (max(highlight_coordinates[0]), highlight_mean[1])
                
            print(COLORS[index], highlight_mean)
        highlight_mean = np.array(highlight_mean)
        if len(highlight_mean)>1 and (highlight_mean == highlight_mean).all(): # check that the mean is not NaN
            if version=="max":
                results.append((COLORS[index], highlight_max, highlight_pixels))
            else:
                results.append((COLORS[index], highlight_mean, highlight_pixels))
    
    return results

In [60]:
results = detect_highlight(frame)

blue [273.4 638.8]
red [447.71428571 638.17857143]
red [468.66666667 958.14814815]
red [470.71428571 681.80952381]
red [471.57894737 632.68421053]
red [426.875 637.875]
red [457.5 627.5]
red [456.4 606.6]
red [527.2 585.8]
red [484.28571429 609.71428571]
red [445.15384615 609.69230769]
red [456.53846154 681.15384615]
red [439.  627.5]
red [456.63636364 648.36363636]
red [433.2 660.2]
red [434.2 718.2]
pink [413.42105263 638.89473684]
brown [132.  638.5]
yellow [130.72222222 546.88888889]
green [131.33333333 730.66666667]
white [441.56451613 490.83064516]
black [531.5 638.5]


In [61]:
for r in results:
    cv2.imshow("result red", r[2])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
frame = cv2.imread("../frames/frame_114235_3807799.9999999995.jpg")
results = detect_highlight(frame)
for r in results:
    cv2.imshow("result", r[2])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
for r in results:
    cv2.imshow("result", r[2])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
frame = cv2.imread("../frames/frame_208900_6963300.0.jpg")
results = detect_highlight(frame)
for r in results:
    cv2.imshow("result", r[2])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
frame = cv2.imread("../frames/frame_341672_11389033.333333332.jpg")
results = detect_highlight(frame)
for r in results:
    cv2.imshow("result", r[2])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
frame = cv2.imread("../frames/few_balls.jpg")
results = detect_highlight(frame)
for r in results:
    cv2.imshow("result", r[2])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [89]:
frame = cv2.imread("../frames/signal-2024-06-10-033005_002.png")
results = detect_highlight(frame)
for r in results:
    cv2.imshow("result", r[2])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

blue [273.125 639.   ]
red [423.94594595 639.05405405]
red [448.75 658.75]
red [457.0952381  648.85714286]
red [457.0952381  671.71428571]
red [439.8 627.9]
red [448.66666667 637.72222222]
red [439.76470588 648.70588235]
red [465.64705882 637.58823529]
red [468.875 692.875]
red [457.4 627.4]
red [431.23076923 638.38461538]
red [471.84615385 656.38461538]
red [457.25 606.75]
red [399.  524.2]
red [448.4 617. ]
pink [421.33333333 638.        ]
brown [132.  638.5]
yellow [130.46875 547.65625]
green [131.5 730.5]
white [ 80.06060606 699.87878788]
black [531.21052632 638.15789474]


In [90]:
results[1][1]

[(425, 639.0540540540541),
 (450, 658.75),
 (459, 648.8571428571429),
 (458, 671.7142857142857),
 (441, 627.9),
 (450, 637.7222222222222),
 (441, 648.7058823529412),
 (468, 637.5882352941177),
 (470, 692.875),
 (459, 627.4),
 (433, 638.3846153846154),
 (473, 656.3846153846154),
 (458, 606.75),
 (400, 524.2),
 (450, 617.0)]

In [179]:
# show the highlights
new = frame.copy()
new[int(results[0][1][0]), int(results[0][1][1])] = [255,0,0]
for r in results[1][1]:
    print(int(r[0]), int(r[1]))
    new[int(r[0]), int(r[1])] = [255,0,0]
for r in results[2:]:
    print(r[1][0], r[1][1])
    new[int(r[1][0]), int(r[1][1])] = [255,0,0]
cv2.imshow("result", new)
cv2.waitKey(0)
cv2.destroyAllWindows()

425 639
450 658
459 648
458 671
441 627
450 637
441 648
468 637
470 692
459 627
433 638
473 656
458 606
400 524
450 617
423 638.0
133 638.5
133 547.65625
132 730.5
84 699.8787878787879
533 638.1578947368421


In [189]:
# Convert the result
def flatten_red_balls(result):
    points = []
    colors = []
    for r in results:
        if type(r[1])==list:
            for point in r[1]:
                points+=[point]
                colors+=[r[0]]
        else:
            points+=[r[1]]
            colors.append(r[0])
    return points, colors

In [190]:
points, colors = flatten_red_balls(results)

In [191]:
COLOR_STR_TO_BGR = {"red": (0,0,255), "blue":(255,0,0), "pink":(130, 130, 255), "green":(0,255,0), 
                    "black":(50,50,50), "brown":(20, 70, 130), "yellow":(0, 190, 250)}
def visualize(frame, points, colors):
    image = np.zeros((720, 1280, 3), np.uint8)
    radius = 5
    thickness = 7
    for point, color in zip(points, colors):
        image = cv2.circle(image, (int(point[1]), int(point[0])), radius, COLOR_STR_TO_BGR.get(color, (255,255,255)), thickness)

    images = np.hstack((frame, image))
    

    cv2.imshow('side by side', images)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [192]:
visualize(frame, points, colors)